# E-commerce business transaction

## import libraries

In [ ]:
import numpy  as np
import pandas as pd

import datetime as dt
import calendar

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from plotly.offline import plot

## load dataset

In [ ]:
path_to_dataset = '/data/sales_transaction.csv'
dataset = pd.read_csv(path_to_dataset)

In [ ]:
df = dataset.copy()

In [ ]:
df.head(10)

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom
1,581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom
2,581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom
3,581475,12/9/2019,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom
4,581475,12/9/2019,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom
5,581475,12/9/2019,21705,Bag 500g Swirly Marbles,10.65,24,13069.0,United Kingdom
6,581475,12/9/2019,22118,Joy Wooden Block Letters,11.53,18,13069.0,United Kingdom
7,581475,12/9/2019,22119,Peace Wooden Block Letters,12.25,12,13069.0,United Kingdom
8,581475,12/9/2019,22217,T-Light Holder Hanging Lace,10.65,12,13069.0,United Kingdom
9,581475,12/9/2019,22216,T-Light Holder White Lace,10.55,24,13069.0,United Kingdom


In [ ]:
df.shape

(536350, 8)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536350 entries, 0 to 536349
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionNo  536350 non-null  object 
 1   Date           536350 non-null  object 
 2   ProductNo      536350 non-null  object 
 3   ProductName    536350 non-null  object 
 4   Price          536350 non-null  float64
 5   Quantity       536350 non-null  int64  
 6   CustomerNo     536295 non-null  float64
 7   Country        536350 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 32.7+ MB


**about the dataset**

This dataset is a sales transactions of an online store in the UK. 

the dataset contains over 500k transactions and 8 columns as listed below:

- TransactionNo (categorical): a six-digit unique number that defines each transaction. The letter “C” in the code indicates a cancellation.

- Date (numeric): the date when each transaction was generated.

- ProductNo (categorical): a five or six-digit unique character used to identify a specific product.

- Product (categorical): product/item name.

- Price (numeric): the price of each product per unit in pound sterling (£).

- Quantity (numeric): the quantity of each product per transaction. Negative values related to cancelled transactions.

- CustomerNo (categorical): a five-digit unique number that defines each customer.

- Country (categorical): name of the country where the customer resides


## preprocess dataset

In [ ]:
# check for NaN values
df.isna().sum()

TransactionNo     0
Date              0
ProductNo         0
ProductName       0
Price             0
Quantity          0
CustomerNo       55
Country           0
dtype: int64

because the ratio of NaN values to the total number of entries is not high, dropping them instead of imputing them.

In [ ]:
# drop NaN values
df.dropna(inplace=True)

In [ ]:
df.isna().sum()

TransactionNo    0
Date             0
ProductNo        0
ProductName      0
Price            0
Quantity         0
CustomerNo       0
Country          0
dtype: int64

In [ ]:
df.columns

Index(['TransactionNo', 'Date', 'ProductNo', 'ProductName', 'Price',
       'Quantity', 'CustomerNo', 'Country'],
      dtype='object')

In [ ]:
# change column names
col_names = ['transaction_no', 'date', 'product_no', 'product_name', 'price',
       'quantity', 'customer_no', 'country']

df.columns = col_names

In [ ]:
# price data validation
print('number of entries with negative price', 
      len(df['price'].loc[df['price'] < 0]))

number of entries with negative price 0


In [ ]:
# countries data validation
df['country'].unique()

array(['United Kingdom', 'Norway', 'Belgium', 'Germany', 'France',
       'Austria', 'Netherlands', 'EIRE', 'USA', 'Channel Islands',
       'Iceland', 'Portugal', 'Spain', 'Finland', 'Italy', 'Greece',
       'Japan', 'Sweden', 'Denmark', 'Cyprus', 'Malta', 'Switzerland',
       'Australia', 'Czech Republic', 'Poland', 'Hong Kong', 'Singapore',
       'RSA', 'Israel', 'Unspecified', 'United Arab Emirates', 'Canada',
       'European Community', 'Bahrain', 'Brazil', 'Saudi Arabia',
       'Lebanon', 'Lithuania'], dtype=object)

In [ ]:
# convert Date to Datetime
date_format = '%m/%d/%Y'
df['date'] = pd.to_datetime(df['date'], format=date_format, errors='coerce')

In [ ]:
# create new columns for 'total money' , 'year' & 'month'
df['total_money'] = df['price'] * df['quantity']
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month

In [ ]:
# year data validation
df['year'].unique()

array([2019, 2018])

In [ ]:
# month data validation
df['month'].unique()

array([12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1])

In [ ]:
# product number data validation 
# length must be 5 or 6
df['product_no'].apply(lambda x :'True' if (len(x)==6) | (len(x)==5) else 'False').unique()

array(['True'], dtype=object)

In [ ]:
# product name data validation
# length of unique product number is same as unique product name
print(len(df['product_no'].unique()) == len(df['product_name'].unique()))

True


## EDA 

In [ ]:
df.head()

,transaction_no,date,product_no,product_name,price,quantity,customer_no,country,total_money,year,month
0,581482,2019-12-09,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom,257.64,2019,12
1,581475,2019-12-09,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom,383.40,2019,12
2,581475,2019-12-09,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom,138.36,2019,12
3,581475,2019-12-09,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom,127.80,2019,12
4,581475,2019-12-09,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom,71.64,2019,12


In [ ]:
# unique number of transactions
print('total number of unique transactions : ', len(df['transaction_no'].unique()))

# unique number of products
print('total number of unique products : ', len(df['product_no'].unique()))

# unique number of customers
print('total number of unique customers : ', len(df['customer_no'].unique()))

# unique number of countries
print('total number of unique countries : ', len(df['country'].unique()))

total number of unique transactions :  23168
total number of unique products :  3767
total number of unique customers :  4738
total number of unique countries :  38


### customer analysis

In [ ]:
num_cancelled_trans = len(df['quantity'].loc[df['quantity'] < 0])
num_total_transactions = len(df['quantity'])

ratio_cancelled_transactions = round((
    num_cancelled_trans / num_total_transactions) * 100, 2)
percentage_cancelled_transactions = str(ratio_cancelled_transactions)+ '%'

print('ratio of cancelled transactions to total transactions is: ', 
      '\n', percentage_cancelled_transactions)

ratio of cancelled transactions to total transactions is:  
 1.59%


top 10 customers that give highest total money

In [ ]:
list_customer = df['customer_no'].unique()
list_total_money = []

# get total money for each customer
for customer in list_customer:
  total_money = round(
      df['total_money'].loc[df['customer_no'] == customer].sum(), 0)
  list_total_money.append(total_money)

# dictionary to map each customer and it's total money
dic_customer_money = {'customer': list_customer, 'total_money': list_total_money}

# sorted dataframe of customers with total money
df_customer_money = pd.DataFrame(
    dic_customer_money).sort_values(
        'total_money', ascending=False).reset_index(drop=True)

In [ ]:
# chart of top 10 customers with highest total money
pio.templates.default = 'none'
fig = px.bar(
    df_customer_money, 
    y=df_customer_money[0:10]['total_money'], 
    x=df_customer_money[0:10]['customer'], 
    title='Fig 1. Total Money By Top 10 Customers', 
    labels={'y': 'Total Money Given', 'x': 'Customer Number'})

fig.update_traces(marker_color='rgb(158,202,225)', 
                  marker_line_color='rgb(8,48,107)', 
                  textposition='outside', marker_line_width=1.5, 
                  opacity=0.7)

fig.update_xaxes(type='category')
fig.show()

df_customer_money.head(10)

,customer,total_money
0,14646.0,2108960.0
1,18102.0,897137.0
2,12415.0,895267.0
3,17450.0,876816.0
4,14911.0,873038.0
5,14156.0,683869.0
6,14298.0,634174.0
7,13694.0,630950.0
8,17511.0,627685.0
9,16684.0,518977.0


#### customer / customer

ratio of total money of top 10 customers vs the rest of customers

In [ ]:
# ratio of top 10 customers 
# to the rest of customers
top_10_customers_money = df_customer_money[0:10]
rest_customers_money = df_customer_money[10:]

top_10_cust_total_money = top_10_customers_money['total_money'].sum()
rest_cust_total_money = rest_customers_money['total_money'].sum()
total_cust_money = df_customer_money['total_money'].sum()


ratio_top_10_rest_customer = (
    len(df_customer_money[0:10]) / len(df_customer_money[10:]))
ratio_top_10_rest_customers_money = (top_10_cust_total_money / total_cust_money)

In [ ]:
fig = make_subplots(rows=1, cols=2, 
                    specs=[[{'type': 'pie'}, {'type': 'pie'}]], 
                    subplot_titles=['Population Ratio', 'Total Money Ratio'])

labels_1 = ['top 10 population', 'the rest population']
values_1 = [10, len(df_customer_money[10:])]

labels_2 = ['top 10 total money', 'the rest total money']
values_2 = [top_10_cust_total_money, rest_cust_total_money]

fig.add_trace(go.Pie(values=values_1, labels=labels_1), row=1, col=1)
fig.add_trace(go.Pie(values=values_2, labels=labels_2), row=1, col=2)

fig.update_layout(title_text='Fig 2. Top 10 Customers (based on total money) vs The Rest')

fig = go.Figure(fig)
fig.show()

print('Ratio of population: top_10 vs rest', ratio_top_10_rest_customer)
print('Ratio of total money: top_10 vs rest', ratio_top_10_rest_customers_money)

Ratio of population: top_10 vs rest 0.0021150592216582064
Ratio of total money: top_10 vs rest 0.14506105140622372


### country analysis

top 10 countries that have highest amount of transactions

In [ ]:
list_countries = df['country'].unique()
list_total_money = []

# get total money for each country
for country in list_countries:
  total_money = round(
      df['total_money'].loc[df['country'] == country].sum(), 2)
  list_total_money.append(total_money)

# dictionary to map each country and it's total money
dic_country_money = {
    'country':list_countries, 'total_money':list_total_money}

# sorted dataframe of countries with total money
df_country_money = pd.DataFrame(
    dic_country_money).sort_values('total_money',
        ascending=False).reset_index(drop=True)

In [ ]:
# plot top 10 countries with highest total money
pio.templates.default = 'none'

fig = px.bar(
    df_country_money, 
    y=df_country_money[0:10]['total_money'],
    x=df_country_money[0:10]['country'],
    labels={'y':'Total Money Given', 'x':'Country'}, 
    title='Fig 3. Total Money Given By Each Country')

fig.update_traces(
    marker_color='rgb(158,202,225)', 
    marker_line_color='rgb(8,48,107)', 
    marker_line_width=2, textposition='outside', opacity=0.7)

fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_xaxes(type='category')
fig.show()

df_country_money.head(10)

,country,total_money
0,United Kingdom,50011861.05
1,Netherlands,2147811.39
2,EIRE,1660645.07
3,Germany,1350265.40
4,France,1316880.98
5,Australia,988756.35
6,Sweden,396042.61
7,Switzerland,358423.61
8,Japan,283293.47
9,Belgium,271346.98


top 5 countries that have lowest amount transactions

In [ ]:
df_country_money.tail()

,country,total_money
33,Lebanon,5692.32
34,Brazil,4652.27
35,RSA,4259.83
36,Bahrain,3226.86
37,Saudi Arabia,903.15


### year & month analysis

In [ ]:
# total money of each month in 2019
list_months = df['month'].loc[df['year'] == 2019].unique()
list_months = sorted(list_months, reverse=False)
month_names = []
list_total_money = []

for month in list_months:
  # get months name
  month_name = calendar.month_abbr[month]
  month_names.append(month_name)

  # get total money each month
  total_money = round(
      df['total_money'].loc[(df['year']==2019) & (df['month'] == month)].sum(), 3)
  list_total_money.append(total_money)

dic_month_money = {
    'month': month_names, 
    'total_money': list_total_money}

df_month_money = pd.DataFrame(dic_month_money)

In [ ]:
fig = px.line(
    df_month_money, x='month', y='total_money', text='month', 
    title='Fig 4. Total Money By Each Month In 2019', 
    labels={'total_money':'Total Money', 'month':'Month'})

fig.update_traces(textposition='bottom right')
fig.show()

df_month_money

,month,total_money
0,Jan,3650369.49
1,Feb,3299906.63
2,Mar,4353695.02
3,Apr,3416122.51
4,May,4530971.65
5,Jun,4412858.70
6,Jul,4524332.08
7,Aug,4618197.29
8,Sep,6542877.20
9,Oct,6971443.79


#### country / month / total money


top 3 countries with highest total money in each month

In [ ]:
# function to get country and month 
# return total money of the country in the month
def get_tot_money_coun_month(country, month):
  tot_mon_coun_month = df['total_money'].loc[(
      df['country'] == country) & (
          df['year'] == 2019) &(df['month']==month)].sum()

  tot_mon_coun_month = round(tot_mon_coun_month, 2)
  return tot_mon_coun_month

# function to get country and month
# return top 3 countries with highest
# total money of the month
def get_top_coun_tot_money_month(month):
  top_three_countries = []
  list_total_money = []
  for country in list_countries:
    total_money_country_month = get_tot_money_coun_month(country, month)
    list_total_money.append([month, total_money_country_month, country])
  
  top_three_countries = sorted(
      list_total_money, key=lambda l:l[1], reverse=True)[0:3]
  
  return top_three_countries

In [ ]:
# get top 3 countries with highest 
# total money for each month in 2019
top_three_countries_months = []

for month in list_months:
  top_three_countries = get_top_coun_tot_money_month(month)
  top_three_countries_months.append(top_three_countries)

df_top_three_countries_months = pd.DataFrame(
    top_three_countries_months)

df_top_three_countries_months.columns=[['rank_1', 'rank_2', 'rank_3']]

In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']

month_1 = df_top_three_countries_months.iloc[0, :]
month_2 = df_top_three_countries_months.iloc[1, :]
month_3 = df_top_three_countries_months.iloc[2, :]
month_4 = df_top_three_countries_months.iloc[3, :]
month_5 = df_top_three_countries_months.iloc[4, :]
month_6 = df_top_three_countries_months.iloc[5, :]

fig = go.Figure()
fig.add_trace(go.Bar(
    x=months,
    y=[month_1[0][1],month_2[0][1], month_3[0][1], month_4[0][1], month_5[0][1], month_6[0][1]],
    name='rank_1',
    text=[month_1[0][2],month_2[0][2], month_3[0][2], month_4[0][2], month_5[0][2], month_6[0][2]],
    marker_color='aquamarine'
))

fig.add_trace(go.Bar(
    x=months,
    y=[month_1[1][1],month_2[1][1], month_3[1][1], month_4[1][1], month_5[1][1], month_6[1][1]],
    name='rank_2',
    marker_color='darkgoldenrod', 
    text=[month_1[1][2],month_2[1][2], month_3[1][2], month_4[1][2], month_5[1][2], month_6[1][2]],
))

fig.add_trace(go.Bar(
    x=months,
    y=[month_1[2][1],month_2[2][1], month_3[2][1], month_4[2][1], month_5[2][1], month_6[2][1]],
    name='rank_2',
    marker_color='cadetblue',
    text=[month_1[2][2],month_2[2][2], month_3[2][2], month_4[2][2], month_5[2][2], month_6[2][2]],
))

fig.update_layout(
    barmode='stack', 
    title_text='Fig 5.1. Top 3 Countries With Highest Total Money Per Month')

fig.show()

In [ ]:
month_7 = df_top_three_countries_months.iloc[6, :]
month_8 = df_top_three_countries_months.iloc[7, :]
month_9 = df_top_three_countries_months.iloc[8, :]
month_10 = df_top_three_countries_months.iloc[9, :]
month_11 = df_top_three_countries_months.iloc[10, :]
month_12 = df_top_three_countries_months.iloc[11, :]

months = ['Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=months,
    y=[month_7[0][1],month_8[0][1], month_9[0][1], month_10[0][1], month_11[0][1], month_12[0][1]],
    name='rank_1',
    text=[month_7[0][2],month_8[0][2], month_9[0][2], month_10[0][2], month_11[0][2], month_12[0][2]],
    marker_color='aquamarine'
))

fig.add_trace(go.Bar(
    x=months,
    y=[month_7[1][1],month_8[1][1], month_9[1][1], month_10[1][1], month_11[1][1], month_12[1][1]],
    name='rank_2',
    marker_color='darkgoldenrod', 
    text=[month_7[1][2],month_8[1][2], month_9[1][2], month_10[1][2], month_11[1][2], month_12[1][2]],
))

fig.add_trace(go.Bar(
    x=months,
    y=[month_7[2][1],month_8[2][1], month_9[2][1], month_10[2][1], month_11[2][1], month_12[2][1]],
    name='rank_2',
    marker_color='cadetblue',
    text=[month_7[2][2],month_8[2][2], month_9[2][2], month_10[2][2], month_11[2][2], month_12[2][2]],
))

fig.update_layout(
    barmode='stack', 
    title_text='Fig 5.2. Top 3 Countries With Highest Total Money Per Month',)

fig.show()

#### country / month

total money per month that is recieved from top 10 countries with highest total money

In [ ]:
list_top_10_countries = list(df_country_money.loc[0:10, 'country'])
list_top_country_money_per_month = []

for country in list_top_10_countries:
  for month in list_months:
    total_month = round(
        df['total_money'].loc[((df['country'] == country) & (
            df['month']==month) & (df['year']==2019))].sum(), 2)
    list_top_country_money_per_month.append([month, country, total_month])

df_top_country_money_per_month = pd.DataFrame(
    list_top_country_money_per_month, 
    columns=['month', 'country', 'total_money']).reset_index(drop=True)

In [ ]:
fig = px.line(df_top_country_money_per_month[0:12], 
    x='month', y='total_money', color='country', markers=True, 
    title='Fig 6.1. Total Money Per Month For The UK', 
    labels={'month':'Month', 'total_money':'Total Money'})

fig.show()

In [ ]:
fig = px.line(df_top_country_money_per_month[12:], 
    x='month', y='total_money', color='country', 
    title='Fig 6.2. Total Money Per Month For Top Countries With Highest Total Money (except UK)', 
    labels={'month': 'Month', 'total_money':'Total Money'})

fig.show()

### product analysis

#### product / total money

top 10 products that give highest total money

In [ ]:
list_products = df['product_no'].unique()
list_product_names = []
list_total_money = []

# get product name & total money for each product
for product in list_products:
  # append product name
  product_name = df['product_name'].loc[df['product_no'] == product].unique()[0]
  list_product_names.append(product_name)
  
  # append unique product total money
  total_money = df['total_money'].loc[df['product_no'] == product].sum()
  list_total_money.append(total_money)
  

# dictionary of product name & total money for each product
dic_product_money = {
    'product_number':list_products,
    'product_name': list_product_names, 
    'total_money':list_total_money}

# dataframe of product and total money
df_product_money = pd.DataFrame(
    dic_product_money).sort_values(
        'total_money', ascending=False).reset_index(drop=True)

In [ ]:
# plot top 10 products  with highest total money
pio.templates.default = 'none'

fig = px.bar(
    df_product_money, 
    y=df_product_money[0:10]['total_money'],
    x=df_product_money[0:10]['product_number'],
    color=df_product_money[0:10]['product_number'],
    text=df_product_money[0:10]['product_name'],
    labels={'y':'Total Money Given', 'x':'Product Number'}, 
    title='Fig 7. Total Money Given By Each Product')

fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_xaxes(type='category')
fig.show()

In [ ]:
df_product_money.head(10)

,product_number,product_name,total_money
0,22197,Popcorn Holder,583286.51
1,84077,World War 2 Gliders Asstd Designs,557170.43
2,23843,Paper Craft Little Birdie,501359.05
3,85123A,Cream Hanging Heart T-Light Holder,451401.70
4,84879,Assorted Colour Bird Ornament,420743.70
5,21212,Pack Of 72 Retrospot Cake Cases,388424.61
6,23084,Rabbit Night Light,327794.21
7,85099B,Jumbo Bag Red Retrospot,290474.52
8,22423,Regency Cakestand 3 Tier,289288.60
9,22492,Mini Paint Set Vintage,284870.92


top 5 products that bring lowest total money
(excluding cancelled orders)

In [ ]:
df_product_money.loc[df_product_money['total_money'] > 0].tail()

,product_number,product_name,total_money
3738,84227,Hen House W Chick In Nest,10.68
3739,23609,Set 10 Cards Snowy Robin 17099,6.19
3740,84550,Crochet Lilac/Red Bear Keyring,6.19
3741,23630,Set 10 Cards Hanging Baubles 17080,6.19
3742,85047,White Beaded Garland String 20light,0.82


top 5 products that bring lowest total money (including cancelled orders)

In [ ]:
df_product_money.tail()

,product_number,product_name,total_money
3762,85063,Cream Sweetheart Magazine Rack,-78.76
3763,20703,Blue Padded Soft Mobile,-87.66
3764,35400,Wooden Box Advent Calendar,-119.90
3765,79323W,White Cherry Lights,-137.36
3766,21645,Assorted Tutti Frutti Round Box,-286.56


#### product / quantity

top most sold products (highest number of quantity)

In [ ]:
list_total_quantities = []

# get list of each product with it's total quantity 
for product in list_products:
  total_quantity = df['quantity'].loc[(
      df['product_no'] == product) & (df['year'] == 2019)].sum()
  list_total_quantities.append(total_quantity)

dic_quntity_product = {
    'product_number': list_products, 
    'product_name': list_product_names,
    'total_quantity': list_total_quantities
}

df_total_quantity_products = pd.DataFrame(
    dic_quntity_product).sort_values(
        by='total_quantity', ascending=False).reset_index(drop=True)

In [ ]:
top_10_product_quantity = df_total_quantity_products.head(10)

fig = px.bar(
    top_10_product_quantity, x='product_number', 
    y='total_quantity', text='product_name',
    color='product_number',
    title='Fig 8. Top 10 Products With Total Quantity Sold in 2019',
    labels={'total_quantity': 'Total Quantity', 'product_number':'Product Number'})
fig.update_xaxes(type='category')

fig.show()

top_10_product_quantity

,product_number,product_name,total_quantity
0,22197,Popcorn Holder,53719
1,84077,World War 2 Gliders Asstd Designs,48652
2,85099B,Jumbo Bag Red Retrospot,45237
3,84879,Assorted Colour Bird Ornament,34074
4,21212,Pack Of 72 Retrospot Cake Cases,32155
5,85123A,Cream Hanging Heart T-Light Holder,32035
6,23084,Rabbit Night Light,30680
7,22492,Mini Paint Set Vintage,23727
8,22616,Pack Of 12 London Tissues,23417
9,21977,Pack Of 60 Pink Paisley Cake Cases,23029


#### product / month

trend of total money of top 10 products with highest total money in every month

In [ ]:
top_10_products_total_money = df_product_money[0:10]
list_top_product_money_per_month = []

# get total money of each product for each month
for product in top_10_products_total_money['product_number']:
  for month in list_months:
    total_money = round(df['total_money'].loc[(
        (df['product_no']==product) & (
            df['year']==2019) & (df['month']==month))].sum(), 2)
    product_name = df['product_name'].loc[df['product_no']==product].unique()[0]
    list_top_product_money_per_month.append(
        [month, product, product_name, total_money])

In [ ]:
df_top_product_money_month = pd.DataFrame(
    list_top_product_money_per_month, 
    columns=['month', 'product_number', 'product_name','total_money']).reset_index(drop=True)

In [ ]:
fig = px.line(df_top_product_money_month[0:60], 
    x='month', y='total_money', color='product_name',
    labels={'month':'Month', 'total_money':'Total Money'},
    title='Fig 9.1. Total Money Per Month For Top Products With Highest Total Money')

fig.show()

In [ ]:
fig = px.line(df_top_product_money_month[60:], 
    x='month', y='total_money', color='product_name',
    labels={'month':'Month', 'total_money':'Total Money'}, 
    title='Fig 9.2. Total Money Per Month For Top Products With Highest Total Money')

fig.show()

#### product / month / quantity

total quantity of top products based on highest total money per month

In [ ]:
list_top_product_quantity_per_month = []

# get total quantity of each product for each month
for product in top_10_products_total_money['product_number']:
  for month in list_months:
    total_quantity = round(df['quantity'].loc[(
        (df['product_no']==product) & (
            df['year']==2019) & (df['month']==month))].sum(), 2)
    product_name = df['product_name'].loc[df['product_no']==product].unique()[0]
    list_top_product_quantity_per_month.append(
        [month, product, product_name, total_quantity])

In [ ]:
df_top_product_quantity_month = pd.DataFrame(
    list_top_product_quantity_per_month, 
    columns=['month', 'product_number', 'product_name','total_quantity'])

In [ ]:
fig = px.line(df_top_product_quantity_month[0:60], 
    x='month', y='total_quantity', color='product_name',
    labels={'month':'Month', 'total_quantity':'Total Quantity'}, 
    title='Fig 10.1. Total Quantity Per Month For Top Products With Highest Total Money')

fig.show()

In [ ]:
# data for the product: Paper Craft Little Birdie
df.loc[df['product_name']=='Paper Craft Little Birdie']

,transaction_no,date,product_no,product_name,price,quantity,customer_no,country,total_money,year,month
142,581483,2019-12-09,23843,Paper Craft Little Birdie,12.38,80995,16446.0,United Kingdom,1002718.10,2019,12
1616,C581484,2019-12-09,23843,Paper Craft Little Birdie,6.19,-80995,16446.0,United Kingdom,-501359.05,2019,12


In [ ]:
fig = px.line(df_top_product_quantity_month[60:], 
    x='month', y='total_quantity', color='product_name',
    labels={'month':'Month', 'total_quantity':'Total Quantity'}, 
    title='Fig 10.2. Total Quantity Per Month For Top Products With Highest Total Money')

fig.show()

#### product / customer 

top 3 products for each top 10 customers

In [ ]:
df_top_customer_product = pd.DataFrame(
    columns=['customer_no', 'product_no', 'product_name', 'total_quantity', 'total_money'])

# get top 3 products for each top 10 customer
for customer in top_10_customers_money['customer']:
  df_product_customer = df.loc[df['customer_no']==customer]
  list_top_customer_product = []
  for product in df_product_customer['product_no'].unique():
    product_name = df['product_name'].loc[
        (df['customer_no']==customer) & (df['product_no']==str(product))].unique()
    
    # get total quantity of each product
    product_total_quantity = df['quantity'].loc[
        (df['customer_no']==customer) & (df['product_no']==str(product))].sum()

    # get total money of each product
    product_total_money = df['total_money'].loc[
        (df['customer_no']==customer) & (df['product_no']==str(product))].sum()
    
    list_top_customer_product.append(
        [customer, product, product_name, product_total_quantity, product_total_money])
  
  # sort based on total money and select top 3 products    
  list_top_customer_top_product = sorted(
      list_top_customer_product, key=lambda l:l[4], reverse=True)[0:3]
  
  # create a dataframe
  df_top_customer_product = df_top_customer_product.append(
      pd.DataFrame(
          list_top_customer_top_product, 
          columns=['customer_no', 'product_no', 'product_name', 'total_quantity', 'total_money']), ignore_index=True)

In [ ]:
fig = px.bar(df_top_customer_product, 
             x='customer_no', y='total_money', color='product_no',
             text='total_quantity', title='Fig 11. Top 3 Products Of Top 10 Customers', 
             hover_data=['customer_no', 'product_name', 'total_quantity', 'total_money'], 
             labels={'customer_no':'Customer Number', 
                     'total_money':'Total Money Of Each Product'})

fig.update_xaxes(type='category')

fig.show()

In [ ]:
df_top_customer_product.head()

,customer_no,product_no,product_name,total_quantity,total_money
0,14646.0,22629,[Spaceboy Lunch Box],4492,52166.20
1,14646.0,23084,[Rabbit Night Light],4801,50522.78
2,14646.0,22630,[Dolly Girl Lunch Box],4096,47802.24
3,18102.0,22189,[Cream Heart Card Holder],5946,72895.62
4,18102.0,22188,[Black Heart Card Holder],4104,49647.02


In [ ]:
# average number of products in each transaction
# (including the cancelled transaction)
total_trans = len(df['transaction_no'].unique())
print('total number of unique transactions are', total_trans)

# total number of quantities
total_quantity = df['quantity'].sum()
print('total number of quantities is',total_quantity)

# average number of quantity
avg_quan_per_trans = int(total_quantity / total_trans)

print('average number of purchased quantities on each transaction is: ', 
      avg_quan_per_trans)

total number of unique transactions are 23168
total number of quantities is 5322139
average number of purchased quantities on each transaction is:  229


In [ ]:
# average number of products in each transaction
# (excluding the cancelled transaction)
total_trans = len(df['transaction_no'].unique())
print('total number of unique transactions are', total_trans)

# total number of quantities
total_quantity = df['quantity'].loc[df['quantity'] > 0].sum()
print('total number of quantities is',total_quantity)

# average number of quantity
avg_quan_per_trans = int(total_quantity / total_trans)

print('average number of purchased quantities on each transaction is: ', 
      avg_quan_per_trans)

total number of unique transactions are 23168
total number of quantities is 5591490
average number of purchased quantities on each transaction is:  241


In [ ]:
# total number of quantities in each transaction
# (including cancelled transactions)
list_total_quan_trans = []
for transaction in df['transaction_no'].unique():
  total_quantity = df['quantity'].loc[df['transaction_no'] == transaction].sum()
  list_total_quan_trans.append([transaction, total_quantity])

df_transaction_quantity = pd.DataFrame(
    list_total_quan_trans, columns=['transaction_no', 'total_quantity'])

In [ ]:
fig = px.line(df_transaction_quantity, 
    x='transaction_no', y='total_quantity', markers=True, 
    title='Fig 12. Total Quantity Per Unique Transaction', 
    labels={'transaction_no':'Transaction Number', 'total_quantity':'Total Quantity'})

fig.show()

## Findings

### Questions to answer 

**1 - How was the sales trend over the months?**

According to Fig 4. the sales trend in the year 2019 starts in January with total money of 3.6M, and it keeps increasing until it reaches its peak in December with total money of 7.7M. Afterwards, the sales fall dramatically to their minimum with a value of just about 2M by the end of December.

**2 - What are the most frequently purchased products?**

According to Fig 7. the top 10 products with the highest total money are: 
- Popcorn Holder: 580K
- World War 2 Gliders Asstd Designs: 550K
- Paper Craft Little Birdie: 500K
- Cream Hanging Heart T-Light Holder: 450K
- Assorted Colour Bird Ornament: 420K
- Pack Of 72 Retrospot Cake Cases: 388K
- Rabbit Night Light: 327K
- Jumbo Bag Red Retrospot: 290K
- Regency Cakestand 3 Tier: 289K
- Mini Paint Set Vintage: 284K

However, this analysis is based on the product price & quantity. For further analysis, the product profitability must be taken into account.

**3 - How many products does the customer purchase in each transaction?**

According to Fig 12. the number of quantities varies in each transaction. However, the average number of quantities in each transaction (including cancelled orders) is 241, omitting the cancelled transaction the average number of quantities in each transaction is 229. 

**4 - What are the most profitable segment customers?**

Based on Fig 2. the top 10 customers of the company represent only 0.002% of the company's consumers but contribute about 15% of the total sales revenue.
 
Moreover based on Fig 7. the most profitable customers are those who buy the products listed in Fig 7. 
 
More generally, according to Fig 3. the most profitable segment customers are from the countries listed in Fig 3.

**5 - Based on your findings, what strategy could you recommend to the business to gain more profit?**

Based on the  findings, below is the list of several recommended strategies to help the company grow sales: 
 
- The trend in Fig 4. and Fig 6.1 shows that the total sales trend of the company has a strong correlation with the total sales in the UK. This is an indication that the buyers from the UK are crucial to the company. Therefore the company could target this market segment with different marketing strategies. 

- Based on Fig 8. the top 10 products of the company bring massive sales for the company. Conceivably the company could use various marketing techniques to maneuver these items. 
 
- According to Fig 4. the sales revenue drops significantly in December. The company could offer various promotions and deals to grow sales this month. According to Fig 5. and Fig 6. the biggest customers of the company are from European countries, which makes sense why the sales drop by the end of the year. The company could target other countries at this time of the year when there are no holidays(Eastern / Middle Eastern countries). 

- There are a number of countries such as Lebanon, Brazil, RSA, Bahrain and Saudi Arabia, which bring the least amount of total money to the company. Therefore either the firm could re-calculate its business with these countries (cost vs profit) or it could conduct market research in these countries to promote commodities in favour of the forenamed countries to boost sales. 
 
- Fig 5.1. and Fig 5.2. show the top countries that have which bring the most total money. The company could use various marketing strategies, or announce new items in favour of these countries in order to dominate their market. 
 
- Based on Fig 1. and Fig 2. the top 10 customers of the company contribute about 15% of the total sales of the company. The company could perhaps create a customer loyalty program to incentivize its loyal customers. This could include providing member discounts, outstanding offers or VIP events. 